Before you run this code you will need to download the 'imagenet-vgg-verydeep-19.mat' file and put it to the same folder with this file.
You can download it from here: 
https://dl.dropboxusercontent.com/u/108721752/imagenet-vgg-verydeep-19.mat

In [ ]:
import scipy.io
import scipy.misc
import tensorflow as tf
import numpy as np
import time
from IPython.display import Image
from functools import reduce

In [ ]:
import keras

from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
content_file_name = '1-content.jpg'
style_file_name = '1-style.jpg'

In [ ]:
data = scipy.io.loadmat('./imagenet-vgg-verydeep-19.mat')

image_content = scipy.misc.imread(content_file_name)
#image_content= scipy.misc.imresize(image_content,(224,224))
plt.figure(figsize=(11,11))
plt.imshow(image_content)
image_content = image_content.astype('float32')
image_content = np.ndarray.reshape(image_content,((1,) + image_content.shape)) # 1 means batch_size
#image_content = np.ndarray.reshape(image_content.shape + (1,)) it is needed to handle gray scale image 


In [ ]:
image_style = scipy.misc.imread(style_file_name)
#image_style= scipy.misc.imresize(image_style,(224,224))
plt.figure(figsize=(11,11))
plt.imshow(image_style)
image_style = image_style.astype('float32')
image_style = np.ndarray.reshape(image_style,((1,) + image_style.shape)) # 1 means batch_size
#image_style = np.ndarray.reshape(image_style.shape + (1,)) it is needed to handle gray scale image 

In [ ]:
print(image_content.shape)
print(image_style.shape)

In [ ]:
def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)

def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')

def preprocess(image, mean_pixel):
    return (image - mean_pixel).astype('float32')

def unprocess(image, mean_pixel):
    return (image + mean_pixel).astype('float32')

Creating a network with tensorflow:

In [ ]:
def net(input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    weights = data['layers'][0]
    
    net = {}
    current = input_image
    for i, name in enumerate(layers):
        print(i,name)
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
         

            kernels = np.transpose(kernels, (1, 0, 2, 3))
            #print('kernel:', kernels.shape)
            #print('bias:', bias.shape)
            
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
            #print('current:', current)
            
        elif kind == 'relu':
            current = tf.nn.relu(current)
            #print('current:', current)
        elif kind == 'pool':
            current = _pool_layer(current)
            #print('current:', current)
        net[name] = current
        #print(' ')

    assert len(net) == len(layers)
    return net#, mean_pixel

Creating the same network with keras

In [ ]:
def keras_model(input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    weights = data['layers'][0]
    
    model = Sequential()
    
    current = input_image
    for i, name in enumerate(layers):
        print(i,name)
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            print(kernels.shape, bias.shape)
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
                 
            #kernels_keras = np.transpose(kernels, (3,2,1, 0))
            kernels_keras = np.transpose(kernels, (1,0,2,3))
           
            if i==0:
                model.add(ZeroPadding2D((1,1),data_format="channels_first",input_shape=(input_image.shape[3],input_image.shape[1],input_image.shape[2])))
            else:
                model.add(ZeroPadding2D((1,1),data_format="channels_first"))
            
            print(model.layers[i].input)
            print(model.layers[i].output)
            model.add(Conv2D(bias.size, (3, 3), name=name, data_format="channels_first", weights=[kernels_keras,bias[0]])) 
            
            #model.add(Conv2D)
            #keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, 
            #                    dilation_rate=(1, 1), 
            #                    activation=None, use_bias=True, kernel_initializer='glorot_uniform', 
            #                    bias_initializer='zeros',
            #                    kernel_regularizer=None, 
            #                    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
            #                    bias_constraint=None)
        elif kind == 'relu':
            model.add(Activation('relu', name=name))
        elif kind == 'pool':
            model.add(MaxPooling2D((2,2), data_format="channels_first", name=name, border_mode='same', strides=(2,2)))
             
    return model 

Calculating the mean pixel values of the VGG network. We will use these values for normalization.

In [ ]:
data['normalization'][0][0][0].shape

In [ ]:
mean = data['normalization'][0][0][0]
mean_pixel = np.mean(mean, axis=(0, 1))

In [ ]:
mean_pixel

Let us preprocess the content image

In [ ]:
content_pre = preprocess(image_content, mean_pixel)

Calculate the activities of the VGG net on the content_pre image:

In [ ]:
content_pre.shape

In [ ]:
content_keras_model = keras_model(content_pre)

In [ ]:
content_keras_model

In [ ]:
content = net(content_pre)

In [ ]:
content_keras_model.summary()

In [ ]:
#plot_model(content_keras_model, show_shapes=True, to_file='model.png')
#Image("model.png")

Select the layers that will be used to get the "content features"

In [ ]:
# CONTENT_LAYER = 'relu4_2'
# content_features = {}

# with tf.Session() as sess:
#     content_pre = preprocess(image_content, mean_pixel)
#     content_net = net(np.squeeze(data['layers']), content_pre)
#     content_features[CONTENT_LAYER] = content_net[CONTENT_LAYER].eval()

CONTENT_LAYERS = ('conv1_1', 'conv2_1', 'conv4_1', 'conv4_2')
#CONTENT_LAYERS = ('relu4_2',)
content_features = {}
content_keras_features ={}




In [ ]:

for layer in CONTENT_LAYERS:
    # create a temporal subnetwork up to the actual layer 
    tmpmodel=Model(input=content_keras_model.input, output=content_keras_model.get_layer(layer).output) 
    
    # calculate the values of the "content features" features on the content image
    content_keras_features[layer]=np.transpose(tmpmodel.predict(np.transpose(content_pre,(0,3,1,2)) ),(0,2,3,1))
    print('content_keras_feaures shape:', layer, content_keras_features[layer].shape)
   

In the new image we create, we want to match these content features:

In [ ]:
content_features=content_keras_features

Select the layers that will be used to get the "style features"

In [ ]:
#STYLE_LAYERS = ('conv3_1','conv5_1')
STYLE_LAYERS = ('relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1')

style_pre = preprocess(image_style, mean_pixel)
style_keras_model = keras_model(style_pre)

style_keras_features ={}





In [ ]:
for layer in STYLE_LAYERS:
    
    # create a temporal subnetwork up to the actual layer 
    tmpmodel=Model(input=style_keras_model.input, output=style_keras_model.get_layer(layer).output) 
    
    # calculate the values of the "style features" features on the style image
    features=np.transpose(tmpmodel.predict(np.transpose(style_pre,(0,3,1,2)) ),(0,2,3,1))
    print('features shape:', layer, features.shape)
    
    # rehape the features
    features = np.reshape(features, (-1, features.shape[3]))
    print('reshaped features shape:', layer, features.shape)
    
    # Create a covraince matrix (Gram matrix) from these features
    gram = np.matmul(features.T, features) / features.size
    style_keras_features[layer] = gram
    print('gram shape:', layer, gram.shape)
    print('**************')
        

In the new image we create, we want to match these style features:

In [ ]:
style_features=style_keras_features

In [ ]:
# make stylized image using backpropogation
initial = None
#initial = scipy.misc.imread('./images/cat.jpg')
if initial is None:
    noise = np.random.normal(size=image_content.shape, scale=np.std(image_content) * 0.1)
    initial = tf.random_normal(image_content.shape) * 0.256
else:
    initial = np.array([preprocess(initial, mean_pixel)])
    initial = initial.astype('float32')

Create a new image initialized with the above value. This will be the tensorflow variable that we will update with backpropagation.

In [ ]:
image = tf.Variable(initial)
image_net = net(image)


Set up the parameters. Feel free to change them.

In [ ]:
#content_weight = 5e0
#style_weight = 1e4
#tv_weight = 1e3
#learning_rate = 1e0

content_weight= 5e0
style_weight= 1e2
tv_weight = 1e2
learning_rate = 1e1

iterations =  1000
checkpoint_iterations = 20
print_iterations = 10


Create the content loss function. 

It measure how much the content features of the tensorflow variable "image" deviate from the content features of the content image.

In [ ]:
# content loss
# content_loss = content_weight * (2 * tf.nn.l2_loss(
#         image_net[CONTENT_LAYER] - content_features[CONTENT_LAYER]) / 
#         content_features[CONTENT_LAYER].size)
content_loss = 0
content_losses = []
for content_layer in CONTENT_LAYERS:
    content_losses.append(2 * tf.nn.l2_loss(
                          image_net[content_layer] - content_features[content_layer]) / 
                          content_features[content_layer].size)
content_loss += content_weight * reduce(tf.add, content_losses)

In [ ]:
content_loss

Create the style loss function.
It measure how much the style Gram matrices of the tensorflow variable "image" deviate from the style Gram matrices of the style image.

In [ ]:
# style loss
style_loss = 0
style_losses = []
for style_layer in STYLE_LAYERS:
    layer = image_net[style_layer]
    _, height, width, number = map(lambda i: i.value, layer.get_shape())
    size = height * width * number
    feats = tf.reshape(layer, (-1, number))
    gram = tf.matmul(tf.transpose(feats), feats) / size
    style_gram = style_features[style_layer]
    style_losses.append(2 * tf.nn.l2_loss(gram - style_gram) / style_gram.size)
style_loss += style_weight * reduce(tf.add, style_losses)

In [ ]:
style_loss

In [ ]:
def _tensor_size(tensor):
    from operator import mul
    return reduce(mul, (d.value for d in tensor.get_shape()), 1)

In [ ]:
# total variation denoising
tv_y_size = _tensor_size(image[:,1:,:,:])
tv_x_size = _tensor_size(image[:,:,1:,:])
tv_loss = tv_weight * 2 * (
        (tf.nn.l2_loss(image[:,1:,:,:] - image[:,:image_content.shape[1]-1,:,:]) /
            tv_y_size) +
        (tf.nn.l2_loss(image[:,:,1:,:] - image[:,:,:image_content.shape[2]-1,:]) /
            tv_x_size))

In [ ]:
print(image)

The final loss function is the sum of the content_loss, style_loss, and tv_loss functions.

In [ ]:
# overall loss
loss = content_loss + style_loss + tv_loss

Set Adam to minimize the final loss function

In [ ]:
# optimizer setup
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
def imsave(path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    scipy.misc.imsave(path, img)

OPTIMIZATION STARTS NOW

In [ ]:
# optimization
best_loss = float('inf')
best = None

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer)
    for i in range(iterations):
        print(i)
        
        # the next step is the actual tensorflow training/minimization step
        train_step.run()
        
        # save the images after some iterations
        if i % checkpoint_iterations == 0 or i == iterations - 1:
            this_loss = loss.eval()
            if this_loss < best_loss:
                best_loss = this_loss
                
                # this is the best image so far
                best = image.eval()
                print("new minimum found")
                newimg= unprocess(best.reshape(image_content.shape[1:]), mean_pixel)
                img = np.clip(newimg, 0, 255).astype(np.uint8)
                plt.figure(figsize=(11,11))
                plt.imshow(img)
                plt.show()
                
            
            # save a check point
            import os
            try:
                os.makedirs('./checks/'+str.split(content_file_name,'.')[0])
            except OSError:
                pass
            timestr = time.strftime("%Y%m%d_%H%M%S")
            filename_cp = './checks/'+str.split(content_file_name,'.')[0]+'/'+timestr+'.jpg'
            
            cp = unprocess(best.reshape(image_content.shape[1:]), mean_pixel)
            imsave(filename_cp, cp)
           
        
        if i % print_iterations == 0 or i == iterations - 1:
            print('Iteration %d/%d' % (i + 1, iterations))
            print('  content loss: %g' % content_loss.eval())
            print('    style loss: %g' % style_loss.eval())
            print('       tv loss: %g' % tv_loss.eval())
            print('    total loss: %g' % loss.eval())

    output = unprocess(best.reshape(image_content.shape[1:]), mean_pixel)

In [ ]:
imsave('output_'+content_file_name, output)

In [ ]:
Image(filename = 'output_'+content_file_name)